In [38]:
import pandas as pd

from underthesea import word_tokenize
from torchtext.data.utils import get_tokenizer
from typing import Iterable, List
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
import warnings
warnings.filterwarnings('ignore')
import pickle
import tensorboard

In [2]:
df = pd.read_csv('data/train/eng-vie.csv')
df.head()


,English,Vietnamese
0,rachel pike : the science behind a climate hea...,khoa học đằng sau một tiêu đề về khí hậu\n
1,"in 4 minutes , atmospheric chemist rachel pike...","trong 4 phút , chuyên gia hoá học khí quyển ra..."
2,i &apos;d like to talk to you today about the ...,tôi muốn cho các bạn biết về sự to lớn của nhữ...
3,headlines that look like this when they have t...,có những dòng trông như thế này khi bàn về biế...
4,they are both two branches of the same field o...,cả hai đều là một nhánh của cùng một lĩnh vực ...


In [39]:
%load_ext tensorboard
%tensorboard --logdir runs/Mar26_06-13-08_7ff693238ec8

In [3]:
import pickle

SRC_LANGUAGE = 'English'
TGT_LANGUAGE = 'Vietnamese'

def vi_tokenizer(sentence):
    tokens = word_tokenize(sentence)
    return tokens

# Place-holders
token_transform = {}
vocab_transform = {}

token_transform[SRC_LANGUAGE] = get_tokenizer('basic_english')
token_transform[TGT_LANGUAGE] = get_tokenizer(vi_tokenizer)

# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    for index,data_sample in data_iter:
        yield token_transform[language](data_sample[language])


# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']
# Load tokenizer và vocab từ file
with open('models/token_vocab_data.pkl', 'rb') as f:
    token_transform, vocab_transform = pickle.load(f)

In [4]:
for language in ['English', 'Vietnamese']:
    print(f"{language} vocabulary size: {len(vocab_transform[language])}")

English vocabulary size: 52051
Vietnamese vocabulary size: 50359


In [5]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #Check whether running on gpu or cpu

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float = 0.1,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()

        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [6]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [8]:

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])

SRC_VOCAB_SIZE, TGT_VOCAB_SIZE

(52051, 50359)

In [7]:
torch.manual_seed(22)
SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8 # embed_dim must be divisible by num_heads
FFN_HID_DIM = 512
BATCH_SIZE = 32
NUM_ENCODER_LAYERS = 4
NUM_DECODER_LAYERS = 4
DROP_OUT = 0.1

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM,DROP_OUT)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)


In [8]:
# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# src and tgt language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor

In [9]:
# Load trạng thái của mô hình từ file
state_dict = torch.load("models/viEn_transformer.pth" ,map_location=torch.device('cpu'))

# Khôi phục trạng thái của mô hình từ state_dict
transformer.load_state_dict(state_dict)

<All keys matched successfully>

In [10]:
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE).long()
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [40]:
input_sentence = input('Enter an English sentence: ')

# Print input and actual translation
print('Input English sentence:', input_sentence)

# Call the translate function here with the input sentence
predicted_translation = translate(transformer, input_sentence)

# Print predicted translation
print('Predicted Vietnamese Translation:', predicted_translation)
print("\n")

Input English sentence: Hello, how are you?
Predicted Vietnamese Translation:  xin chào làm bạn thế nào hạ sinh một chiếc bánh 



Tất nhiên, dưới đây là 20 câu tiếng Anh cùng với bản dịch tiếng Việt:

1. Hello, how are you?
Xin chào, bạn có khỏe không?

2. I love reading books.
Tôi yêu thích đọc sách.

3. She plays the piano beautifully.
Cô ấy chơi piano đẹp đấy.

4. Have you finished your homework yet?
Bạn đã hoàn thành bài tập về nhà chưa?

5. We are going to the beach tomorrow.
Chúng tôi sẽ đi biển vào ngày mai.

6. The cat is sleeping on the couch.
Con mèo đang ngủ trên ghế sofa.

7. He wants to learn how to cook.
Anh ấy muốn học cách nấu ăn.

8. My favorite color is blue.
Màu yêu thích của tôi là màu xanh.

9. They live in a big house near the park.
Họ sống trong một căn nhà lớn gần công viên.

10. Did you watch the new movie last night?
Bạn đã xem bộ phim mới vào tối qua chưa?

11. I can speak English and French.
Tôi có thể nói tiếng Anh và tiếng Pháp.

12. She's studying hard for her exams.
Cô ấy đang học hành chăm chỉ cho kì thi.

13. We went shopping at the mall yesterday.
Chúng tôi đã đi mua sắm tại trung tâm thương mại vào ngày hôm qua.

14. The weather is beautiful today, isn't it?
Thời tiết hôm nay đẹp quá, phải không?

15. He's always late for meetings.
Anh ấy luôn đến muộn trong các cuộc họp.

16. Can you help me with this math problem?
Bạn có thể giúp tôi với bài toán này được không?

17. We had a delicious dinner at the restaurant.
Chúng tôi đã có một bữa tối ngon tại nhà hàng.

18. She's a talented singer and dancer.
Cô ấy là một ca sĩ và vũ công tài năng.

19. My parents are coming to visit us next week.
Bố mẹ tôi sẽ đến thăm chúng tôi vào tuần sau.

20. I'm looking forward to the weekend.
Tôi đang mong chờ cuối tuần đấy.

In [15]:
import pandas as pd

# Giả sử df là DataFrame của bạn
df = pd.read_csv('data/test/english_sentences.csv')
df.iloc[89555:89565]

,Unnamed: 0,English words/sentences
89555,89555,I don't remember that at all.
89556,89556,I don't remember that at all.
89557,89557,I don't see anything special.
89558,89558,I don't see anything strange.
89559,89559,I don't see anything strange.
89560,89560,I don't see how I can refuse.
89561,89561,I don't take orders from you.
89562,89562,I don't take orders from you.
89563,89563,I don't talk to them anymore.
89564,89564,I don't think I can eat this.
